# HSE 2019 ML (bachelors 3rd course)
## Homework #3
### Student: Igor Tukh

In [309]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import nltk
import time

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from collections import defaultdict
from scipy.stats import norm
from sklearn import metrics
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [2]:
DATASETS_DIR = 'datasets'

In [3]:
def load_csv_dataset(dataset_name, label_name='label'):
    dataset_path = os.path.join(DATASETS_DIR, dataset_name)
    if label_name is not None:
        dataset = pd.read_csv(dataset_path)
        data_columns = np.array([name for name in dataset.columns if name != label_name])
        return dataset[data_columns], dataset[np.array([label_name])]
    else:
        return pd.read_csv(dataset_path)

### SVM

#### Dataset: blobs2.csv.

#### (3) Apply an SVM classifier (for example, from sklearn package) to blobs2.csv dataset and draw the result (both the support vectors and the separation hyperplane). Use the following kernels:
* **Linear.**
* **Polynomial, degree 2.**
* **Polynomial, degree 3.**
* **Polynomial, degree 5.**
* **Radial Basis Function.**

Let's implemnet visualisation function based on [sklearn implementation](https://scikit-learn.org/0.19/auto_examples/svm/plot_iris.html)

In [4]:
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

def visulize_SVC(ax, classifier_title, classifier, X, y):
    X0 = X['x'].to_numpy()
    X1 = X['y'].to_numpy()
    
    xx, yy = make_meshgrid(X0, X1)

    plot_contours(ax, classifier, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0.reshape(-1), X1.reshape(-1), c=y.reshape(-1), cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    support_indexes = classifier.support_
    ax.scatter(X0[support_indexes].reshape(-1), X1[support_indexes].reshape(-1), 
               c=y[support_indexes].reshape(-1), cmap=plt.cm.coolwarm, s=40, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title('{}, accuracy: {}'.format(classifier_title, 
                                           classifier.score(X.to_numpy().copy().reshape(-1, 2), y.copy())))

In [5]:
def task1():
    X, y = load_csv_dataset('blobs2.csv')
    y = y.to_numpy().reshape(-1)
    
    classifiers = [SVC(kernel='linear'),
                   SVC(kernel='poly', degree=2, gamma='auto'),
                   SVC(kernel='poly', degree=3, gamma='auto'),
                   SVC(kernel='poly', degree=5, gamma='auto'),
                   SVC(kernel='rbf', gamma='auto')]
    
    classifiers_titles = ['SVC, linear kernel', 
                          'SVC, polynomial kernel (degree = 2)',
                          'SVC, polynomial kernel (degree = 3)',
                          'SVC, polynomial kernel (degree = 5)',
                          'SVC, RBF as kernel']
    
    fig, sub = plt.subplots(3, 3, figsize=(18, 18))

    for classifier, classifier_title, ax in zip(classifiers, classifiers_titles, sub.flatten()):
        classifier.fit(X.to_numpy().copy().reshape(-1, 2), y.copy())
        visulize_SVC(ax, classifier_title, classifier, X, y)
        
    plt.show()
    
task1()

Support vectors in plots above are bigger

#### Datasets: spam.csv and cancer.csv.

#### 2) (3) Apply SVM and RandomForest methods to cancer.csv and spam.csv dataset. Compare training time with close performance on validation (20%)

For determinacy let's select SVC with a linear kernel. For esstimating results let's use $F_1 $score

In [9]:
def get_execution_time(function_to_execute):
    print('Experiment started at {}'.format(time.time()))
    start_time = time.time()
    execution_result = function_to_execute()
    return time.time() - start_time    

def task2():
    spam_X, spam_y = load_csv_dataset('spam.csv')    
    spam_y = spam_y.to_numpy().reshape(-1)
    spam_X = spam_X.to_numpy()
    cancer_X, cancer_y = load_csv_dataset('cancer.csv')
    cancer_y = cancer_y.to_numpy().reshape(-1)
    cancer_y = np.vectorize(lambda label: 0 if label == 'M' else 1)(cancer_y)
    cancer_X = cancer_X.to_numpy()
    spam_X_train, spam_X_test, spam_y_train, spam_y_test = train_test_split(spam_X, spam_y, test_size=0.2)
    cancer_X_train, cancer_X_test, cancer_y_train, cancer_y_test = train_test_split(cancer_X, cancer_y, test_size=0.2)
    
    classifiers = [SVC(kernel='linear'),
                   RandomForestClassifier(n_estimators=100)]
    
    classifiers_names = ['SVM classifier', 'Random forest classifier']
    
    for classifier, classifier_name in zip(classifiers, classifiers_names):
        time = get_execution_time(lambda: classifier.fit(spam_X_train, spam_y_train))
        results = f1_score(spam_y_test, classifier.predict(spam_X_test))
        accuracy = classifier.score(spam_X_test, spam_y_test)
        print('Classifier: {}, time {}, F_1 score {}, accuracy: {}'
              .format(classifier_name, time, results, accuracy))
        
        time = get_execution_time(lambda: classifier.fit(cancer_X_train, cancer_y_train))
        results = f1_score(cancer_y_test, classifier.predict(cancer_X_test))
        accuracy = classifier.score(cancer_X_test, cancer_y_test)
        print('Classifier: {}, time {}, F_1 score {}, accuracy: {}'
              .format(classifier_name, time, results, accuracy))

task2()

Experiment started at 1558172252.604365
Classifier: SVM classifier, time 785.4366760253906, F_1 score 0.9020172910662825, accuracy: 0.9261672095548317
Experiment started at 1558173038.104116
Classifier: SVM classifier, time 1.047410249710083, F_1 score 0.9624060150375939, accuracy: 0.956140350877193
Experiment started at 1558173039.1529632
Classifier: Random forest classifier, time 0.4573092460632324, F_1 score 0.9362962962962964, accuracy: 0.9533116178067318
Experiment started at 1558173039.64484
Classifier: Random forest classifier, time 0.11946320533752441, F_1 score 0.9701492537313432, accuracy: 0.9649122807017544


As we can see, SVM is much slower, with the same (or even worser) scores.

### Bayesian Classifier


#### Datasets: spam.csv and cancer.csv.

#### 3) (6) Implement a Naive Bayesian Classifier with Gaussian features (no need for EM algorithm, sample mean and variance are sufficient for determining the parameters of a feature distribution) and apply it to spam.csv and cancer.csv.  Draw ROC-curves for scaled output probability (the output itself is quite small, so you should scale it in a way that sums up to 1 for a specific datapoint across all classes)


Let's split on train/test (test=20%)

In [326]:
class BayesianClassifier(object):
    def __init__(self):
        self.labels = defaultdict(lambda: 0)
        self.probabilities = defaultdict(lambda: 0)
    
    def fit(self, X, y):
        self.means = {}
        self.variances = {}
        
        for label in y:
            self.labels[label] += 1
        for key, value in self.labels.items():
            self.probabilities[key] = 1. * value / y.shape[0]
        for key in self.labels.keys():
            mask = y == key
            self.means[key] = np.mean(X[mask, :], axis=0)
            self.variances[key] = np.std(X[mask, :], axis=0)
                                        
    def predict_proba(self, X):
        probabilities = np.array([defaultdict(lambda: 0) for _ in X])
        for label in self.labels.keys():
            pr = np.zeros(X.shape, dtype=np.double)
            for ind1 in range(X.shape[0]):
                for ind2 in range(X.shape[1]):
                    loc = self.means[label][ind2]
                    scale = self.variances[label][ind2]
                    pr[ind1][ind2] = norm.pdf(X[ind1][ind2], loc, scale)
                probabilities[ind1][label] = np.prod(pr[ind1]) * self.probabilities[label]
                
        for ind in range(X.shape[0]):
            probabilities_sum = sum(list(probabilities[ind].values()))
            
            if np.isclose(probabilities_sum, 0):
                for key in probabilities[ind].keys():
                    probabilities[ind][key] = 1. / len(self.labels)
            else:
                for key in probabilities[ind].keys():
                    probabilities[ind][key] /= probabilities_sum
        
        
        return probabilities

    def predict(self, X):
        probabilities = self.predict_proba(X)
        results = []
        for ind in range(X.shape[0]):
            max_proba = -1.
            label = None
            for key, value in probabilities[ind].items():
                if max_proba < value:
                    label = key
                    max_proba = value
            results.append(label)
        return np.array(results)
    
    def score(self, X, y, score_function=None):
        y_pred = self.predict(X)
        return score_function(y, y_pred)

In [327]:
def build_roc(X, y, dataset_name='', test_size=0.2):
    clf = BayesianClassifier()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    clf.fit(X_train, y_train)
    
    pr = clf.predict_proba(X_test)
    unique_labels = np.unique(y)
    labels_map = {}
    for ind in range(unique_labels.shape[0]):
        labels_map[unique_labels[ind]] = ind
    
    pos_class = unique_labels[1]
    y_true = np.zeros(y_test.shape[0], dtype=np.int64)
    y_score = np.zeros(y_test.shape[0], dtype=np.double)
    
    for ind in range(X_test.shape[0]):
        y_true[ind] = labels_map[y_test[ind]]
        y_score[ind] = pr[ind][pos_class]
    
    fpr, tpr, _ = metrics.roc_curve(y_true, y_score, labels_map[pos_class])
    
    score = clf.score(X_test, y_test, lambda y1, y2: np.mean(np.array(y1 == y2, dtype=np.int64)))
    print('Datatset {}, test accuracy: {}'.format(dataset_name, score))
    
    plt.clf()
    plt.title('ROC-curve for datatset {}'.format(dataset_name))
    plt.xlabel('fpr')
    plt.ylabel('tpr')
    plt.plot(fpr, tpr)
    plt.show()

In [328]:
spam_X, spam_y = load_csv_dataset('spam.csv')    
build_roc(spam_X.to_numpy(), spam_y.to_numpy().reshape(-1), 'spam')

Datatset spam, test accuracy: 0.8099891422366993


In [244]:
cancer_X, cancer_y = load_csv_dataset('cancer.csv')
build_roc(cancer_X.to_numpy(), cancer_y.to_numpy().reshape(-1), 'cancer')

Datatset cancer, test accuracy: 0.9210526315789473


#### Dataset: smsspam.csv

#### 4) (6) Implement a Naive Bayesian Classifier with discrete features, i.e., numbers for dictionary words in the text in smsspam.csv dataset. So, if your dictionary is N words, each datapoint will have N features. Stemming is advised. Draw ROC-curves for scaled output probability.

In [258]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/itukh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

We also will split data on train/test 80/20

In [286]:
def prepare_dictionaries_with_stemming(X, y):
    words_dictionary = defaultdict(lambda: defaultdict(lambda: 0))
    stemmer = PorterStemmer()
    words = set()
    for ind in range(X.shape[0]):
        current_words = word_tokenize(X[ind])
        current_words = list(map(stemmer.stem, current_words))
        for word in current_words:
            words_dictionary[y[ind]][word] += 1
            words.add(word)
    return words, words_dictionary

In [315]:
def get_probabilities(X, y, words, words_dictionaries):
    probabilities = np.ones((X.shape[0], len(words_dictionaries)), dtype=np.double)
    stemmer = PorterStemmer()
    for label in words_dictionaries.keys():
        total_words = sum([value for value in words_dictionaries[label].values()]) + len(words_dictionaries)
        for ind in range(X.shape[0]):
            current_words = word_tokenize(X[ind])
            current_words = list(map(stemmer.stem, current_words))
            for word in set(current_words):
                if word in words:
                    probabilities[ind][label] *= 1. * (words_dictionaries[label][word] + 1) / total_words
    
    pr = np.zeros(probabilities.shape[0])
    for i in range(probabilities.shape[0]):
        probabilities[i] /= np.sum(probabilities[i])
        pr[i] = probabilities[i][1]
    
    return pr

In [321]:
def build_roc(fpr, tpr, dataset_name):
    plt.clf()
    plt.title('ROC-curve for datatset \'{}\''.format(dataset_name))
    plt.xlabel('fpr')
    plt.ylabel('tpr')
    plt.plot(fpr, tpr)
    plt.show()

In [317]:
def change_labels(y):
    labels = np.unique(y)
    labels_map = {labels[ind]: ind for ind in range(labels.shape[0])}
    return np.vectorize(lambda x: labels_map[x])(y)

In [323]:
def task4():
    X, y = load_csv_dataset('smsspam.csv')
    X = X.to_numpy().reshape(-1)
    y = y.to_numpy().reshape(-1)
    y = change_labels(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    words, words_dictionaries = prepare_dictionaries_with_stemming(X_train, y_train)
    
    pr = get_probabilities(X_test, y_test, words, words_dictionaries)
    y_pred = np.array([0 if pr[i] < 0.5 else 1 for i in range(pr.shape[0])])
    
    fpr, tpr, _ = metrics.roc_curve(y_test, pr, 1)
    build_roc(fpr, tpr, 'SMS spam')
    
    score_f1 = f1_score(y_test, y_pred)
    accuracy = np.mean(np.array(y_test == y_pred, dtype=np.int64))
    print('Dataset: \'SMS spam\', test accuracy: {}, F_1 score: {}'.format(accuracy, score_f1))
    
task4()

Dataset: 'SMS spam', test accuracy: 0.95695067264574, F_1 score: 0.8571428571428572
